In [1]:
#TODO: install streamlit, matplotlib, seaborn>

We have a few ETL functions from our Python EDA that we want to migrate to, for example, a cloud-based setting.

Notable ETL functions include:

- Placing the data in space and time -- northeastern US, starting on 2004,3,23
    - by locating thanksgiving and christmas we could deduce a seasonal 

    - arimax on the sales section level?

    

In [2]:
from database import engine, get_db
from my_url import _SQLALCHEMY_DATABASE_URL
from data_lake import DataLake
import pandas as pd 


2023-10-29 11:53:29,151 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-29 11:53:29,153 INFO sqlalchemy.engine.Engine COMMIT


In [3]:
a = DataLake()

2023-10-29 11:53:29,486 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2023-10-29 11:53:29,486 INFO sqlalchemy.engine.Engine [raw sql] ()
{'campaign_desc': 30,
 'campaign_table': 7208,
 'causal_data': 36786524,
 'coupon': 124548,
 'coupon_redempt': 2318,
 'hh_demographic': 801,
 'hh_summary': 801,
 'product': 92353,
 'transaction_data': 2595732}


In [4]:
a.get_table('hh_summary')

,index,household_key,age_45_plus,income_50K_plus,single_couple_family,has_kids,single,couple,r_score,f_score,...,grain_goods,grocery,home_family,junk_food,kitchen,meat,misc,produce,seasonal,total_sales
0,0,1,1,0,2,0,0,1,5,3,...,668.41,230.24,537.29,764.02,1.59,379.04,303.82,477.82,0.00,4120.07
1,1,7,1,1,2,0,0,1,5,3,...,248.84,141.06,648.67,248.54,0.00,616.74,191.86,371.61,13.46,3170.21
2,2,8,0,0,3,1,0,0,5,4,...,427.40,213.37,901.81,389.36,34.68,956.70,379.84,545.26,15.98,5075.29
3,3,13,0,1,3,1,0,0,5,5,...,611.88,229.27,2771.53,829.34,18.71,604.16,764.12,578.87,70.93,8701.99
4,4,16,1,1,1,0,1,0,4,3,...,279.85,73.24,72.17,127.63,8.07,224.09,137.36,90.22,0.00,1224.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,796,2494,0,1,2,0,0,1,5,1,...,150.75,108.10,281.83,292.02,0.00,374.05,139.68,280.89,1.00,2007.99
797,797,2496,1,1,3,1,0,0,4,2,...,321.16,103.29,424.82,360.46,7.48,1587.79,358.50,332.36,1.79,4015.65
798,798,2497,1,0,1,0,1,0,4,5,...,570.01,350.91,868.09,832.75,3.99,1560.83,529.81,940.00,23.47,6792.55
799,799,2498,0,1,2,0,0,1,5,5,...,319.38,145.57,262.77,252.69,0.00,305.32,128.00,228.36,0.00,2524.85


In [5]:
# startup the API server using `uvicorn main:app --reload` in the shell

# Derived Tables

The code below is being adapted from previous work I did on DunnHumby. 

Because of the data coming out of the db now, some column names need to be lowercased.

The operations below represent a complex combination of data cleaning steps and table joins; there's way too much going on

In [6]:
# Transforming TRANSACTION DATA AND PRODUCT


import datetime as dt
import pandas as pd
import numpy as np



# def hh_summary_create():
# TODO:  ### consider dropping unnecessary columns
### this currently rips directly from the data source, making the data lake obsolete
### 


### TRANSACTION DATA

transactions = pd.read_csv('../data/transaction_data.csv')
#self.get_table('transaction_data')

# filtering empty sales rows..
transactions = transactions[(transactions['QUANTITY'] > 0) & 
                (transactions['SALES_VALUE'] > 0)]

# add datetime values. this should be back-integrated with the data ingestion procedure, now that we have it
day1 = dt.datetime(2004, 3, 23) # as derived in transactions notebook; datetime for 'DAY' == 1
ineedthismany = transactions['DAY'].max()
last = day1 + dt.timedelta(days=int(ineedthismany))
date_range = pd.date_range(day1, last) # date range for our data

# map datetime index to DAY; enumerate() indexes from 0, so we add 1
date_map = {i+1:x for i, x in enumerate(date_range)}
transactions['datetime'] = transactions['DAY'].map(date_map)
# truncate data

# filtering on datetime (data acquisition inconsistency?)
transactions = transactions[(transactions['datetime'] >= "2004-7-1") &
                (transactions['datetime'] < "2006-3-1")]



###  PRODUCT 
prod = pd.read_csv('../data/product.csv')
# Merge
merged = transactions.merge(prod.drop('CURR_SIZE_OF_PRODUCT', axis=1))

# Remove Gasoline Sales
merged.drop(merged[merged['SUB_COMMODITY_DESC']=='GASOLINE-REG UNLEADED'].index, axis=0, inplace=True)
merged.drop(merged[merged['COMMODITY_DESC']=='GASOLINE-REG UNLEADED'].index, axis=0, inplace=True)

# Add section labels
with open('../Section_Labels.txt', 'r') as f:
    product_section_map = eval(f.readlines()[0])
ser = merged['COMMODITY_DESC'].map(product_section_map) # hardcoded;
ser = ser.fillna('misc') # for exceptions ?

merged['Section Labels'] = ser

# Remove one-day transactions -- outliers?
def one_day_transactions(df) -> list:
    no_days = df.groupby('household_key').agg({'DAY':'nunique'})
    return list(no_days[no_days['DAY'] == 1].index)

# remove households with only 1 day of purchases;
merged = merged[~merged['household_key'].isin(one_day_transactions(merged))]    







### Creating Customer-Level Sales by Section Labels
section_dummies = pd.get_dummies(merged['Section Labels'])
#print(section_dummies.sum().sum())
# the transaction-level (item) binary flags for
#section_dummies.sum()

### total sales by section, for each household
### may want to break this out further, into ie. weekly, bi-weekly, monthly, or quarterly sales totals for each section
section_sales = section_dummies.apply(lambda x : x * merged['SALES_VALUE'])
section_sales = section_sales.round(2)

hh_agg = section_sales.join(merged['household_key']).groupby('household_key').agg({col:'sum' for col in section_dummies.columns})

hh_agg['total_sales'] = hh_agg.sum(axis=1)


In [7]:


# ADDING RFM SCORE

last_days = merged.groupby('household_key')['DAY'].max()
R = pd.cut(last_days, [0, 525, 615, 675, 700, np.inf], labels=[1,2,3,4,5], ordered=True).astype('int')
num_baskets = merged.groupby('household_key')['BASKET_ID'].nunique()
grouper = merged.groupby('household_key')['DAY']
days_in_data = grouper.max() - grouper.min() + 1 #(no day 0 in our data)
transactions_per_day = num_baskets/days_in_data
F = pd.qcut(transactions_per_day, 5, labels=[1,2,3,4,5]).astype('int')
M = pd.qcut(np.log(merged.groupby('household_key')['SALES_VALUE'].sum()), 5, labels=[1,2,3,4,5]).astype('int')
temp = pd.concat([R, F, M], axis=1)
temp.columns = ['r_score', 'f_score', 'm_score']
temp['rfm_score'] = temp.sum(axis=1).astype(int)
customer_ranks = pd.cut(temp['rfm_score'], bins=[0,6,9,13,15, np.inf], labels=[1,2,3,4,5], right=False).astype(int)
customer_ranks = pd.DataFrame(customer_ranks) ## THIS IS HACKY BRO
temp['rfm_bins'] = customer_ranks

hh_agg = temp.merge(hh_agg, on='household_key').reset_index()




In [8]:


# DEMOGRAPHIC DATA



demo = pd.read_csv('../data/hh_demographic.csv')
## Alternate Mappings
demo['age_45_plus'] = demo['AGE_DESC'].map({ '19-24':0,
                                        '25-34':0,
                                        "35-44":0,
                                        '45-54':1,
                                        '55-64':1,
                                        '65+':1,
                                        })


demo['income_50K_plus'] = demo['INCOME_DESC'].map({
                        'Under 15K': 0,
                            '15-24K': 0,
                            '25-34K': 0,
                            '35-49K': 0,
                            '50-74K': 1,
                            '75-99K': 1,
                            '100-124K': 1,
                            '125-149K': 1,
                            '150-174K': 1,                   
                            '175-199K': 1,  
                            '200-249K': 1,
                            '250K+': 1,
                        })

# leaving household_size desc IN as a category; single, couple, 3+
demo['single_couple_family'] = demo['HOUSEHOLD_SIZE_DESC'].map({'1':1, '2':2, '3':3,'4':3,'5+':3})
#demo['single_couple_family'] = pd.Categorical(demo['single_couple_family'], 
#                                                [1,2,3,])


demo['has_kids'] = np.where((demo['HH_COMP_DESC'] == '1 Adult Kids') |
                            (demo['HH_COMP_DESC'] == '2 Adults Kids') |
                            (demo['HOUSEHOLD_SIZE_DESC'].isin(['3', '4', '5+'])),
                            1, 0)

demo['single'] =  np.where((demo['HH_COMP_DESC'] == 'Single Female') |
                            (demo['HH_COMP_DESC'] == 'Single Male') |
                            (demo['HOUSEHOLD_SIZE_DESC'] == '1'),
                            1, 0)

demo['couple'] =  np.where((demo['HH_COMP_DESC'] == '2 Adults No Kids'),
                            1, 0)

demo = demo.drop(['AGE_DESC', 'MARITAL_STATUS_CODE', 
            'INCOME_DESC', 'HOMEOWNER_DESC', 'HH_COMP_DESC', 
            'HOUSEHOLD_SIZE_DESC', 'KID_CATEGORY_DESC'], axis=1)


# binary flags for low and high income; low and high age; based on distributions of values in a known data set?

hh_summary = demo.merge(hh_agg, on='household_key')

hh_summary.to_sql(name='hh_summary',
                con=_SQLALCHEMY_DATABASE_URL,
            #      if_exists='append',
                if_exists='replace',
                #chunksize=999,
                method='multi')

hh_summary.to_csv('../data/hh_summary.csv')

In [10]:
# hh_summary_create()


df = a.get_table('hh_summary')
df

,index,household_key,age_45_plus,income_50K_plus,single_couple_family,has_kids,single,couple,r_score,f_score,...,grain_goods,grocery,home_family,junk_food,kitchen,meat,misc,produce,seasonal,total_sales
0,0,1,1,0,2,0,0,1,5,3,...,668.41,230.24,537.29,764.02,1.59,379.04,303.82,477.82,0.00,4120.07
1,1,7,1,1,2,0,0,1,5,3,...,248.84,141.06,648.67,248.54,0.00,616.74,191.86,371.61,13.46,3170.21
2,2,8,0,0,3,1,0,0,5,4,...,427.40,213.37,901.81,389.36,34.68,956.70,379.84,545.26,15.98,5075.29
3,3,13,0,1,3,1,0,0,5,5,...,611.88,229.27,2771.53,829.34,18.71,604.16,764.12,578.87,70.93,8701.99
4,4,16,1,1,1,0,1,0,4,3,...,279.85,73.24,72.17,127.63,8.07,224.09,137.36,90.22,0.00,1224.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,796,2494,0,1,2,0,0,1,5,1,...,150.75,108.10,281.83,292.02,0.00,374.05,139.68,280.89,1.00,2007.99
797,797,2496,1,1,3,1,0,0,4,2,...,321.16,103.29,424.82,360.46,7.48,1587.79,358.50,332.36,1.79,4015.65
798,798,2497,1,0,1,0,1,0,4,5,...,570.01,350.91,868.09,832.75,3.99,1560.83,529.81,940.00,23.47,6792.55
799,799,2498,0,1,2,0,0,1,5,5,...,319.38,145.57,262.77,252.69,0.00,305.32,128.00,228.36,0.00,2524.85
